# CPSC5175 Lab 12 - Chapter 12
#### Jamion Williams

### Problem 12.1b
##### Work upon the UniversalBank.csv dataset.

In [8]:
# necessities

library(caret)
library(MASS)

In [2]:
# Import the dataset and interrogate it.

raw.df <- read.csv("Datasets/UniversalBank.csv",header = TRUE)

str(raw.df)

head(raw.df)

'data.frame':	5000 obs. of  14 variables:
 $ ID                : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Age               : int  25 45 39 35 35 37 53 50 35 34 ...
 $ Experience        : int  1 19 15 9 8 13 27 24 10 9 ...
 $ Income            : int  49 34 11 100 45 29 72 22 81 180 ...
 $ ZIP.Code          : int  91107 90089 94720 94112 91330 92121 91711 93943 90089 93023 ...
 $ Family            : int  4 3 1 1 4 4 2 1 3 1 ...
 $ CCAvg             : num  1.6 1.5 1 2.7 1 0.4 1.5 0.3 0.6 8.9 ...
 $ Education         : int  1 1 1 2 2 2 2 3 2 3 ...
 $ Mortgage          : int  0 0 0 0 0 155 0 0 104 0 ...
 $ Personal.Loan     : int  0 0 0 0 0 0 0 0 0 1 ...
 $ Securities.Account: int  1 1 0 0 0 0 0 0 0 0 ...
 $ CD.Account        : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Online            : int  0 0 0 0 0 1 1 0 1 0 ...
 $ CreditCard        : int  0 0 0 0 1 0 0 1 0 0 ...


ID,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard
<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
6,37,13,29,92121,4,0.4,2,155,0,0,0,1,0


In [4]:
# get rid of zip code and dummy other vars 

drops <- c("ZIP.Code")
pre.df <- raw.df[,!(names(raw.df) %in% drops)]

dummies <- dummyVars(Personal.Loan ~ ., data = pre.df)  

dummy.set <- data.frame(predict(dummies, newdata = pre.df))
dummy.set$Personal.Loan <- as.factor(pre.df$Personal.Loan)
head(dummy.set)

ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities.Account,CD.Account,Online,CreditCard,Personal.Loan
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,25,1,49,4,1.6,1,0,1,0,0,0,0
2,45,19,34,3,1.5,1,0,1,0,0,0,0
3,39,15,11,1,1.0,1,0,0,0,0,0,0
4,35,9,100,1,2.7,2,0,0,0,0,0,0
5,35,8,45,4,1.0,2,0,0,0,0,1,0
6,37,13,29,4,0.4,2,155,0,0,1,0,0


In [6]:
set.seed(420)

trainingList <- createDataPartition(dummy.set$ID, p = 0.6, list = FALSE, times = 1)
train.df <- dummy.set[trainingList,]
test.df <- dummy.set[-trainingList,]

dim(train.df)

dim(test.df)

[1] 3000   13

[1] 2000   13

In [10]:
set.seed(420)

DiscrimFit <- lda(Personal.Loan ~ ., train.df)
DiscrimFit

Call:
lda(Personal.Loan ~ ., data = train.df)

Prior probabilities of groups:
    0     1 
0.907 0.093 

Group means:
        ID      Age Experience   Income   Family    CCAvg Education  Mortgage
0 2510.662 45.63249   20.39765  66.5656 2.389563 1.745491  1.853730  48.95553
1 2406.896 45.20789   19.97491 144.8996 2.637993 3.920072  2.225806 104.84946
  Securities.Account CD.Account    Online CreditCard
0         0.09702315 0.03711871 0.6027196  0.2958471
1         0.13261649 0.31541219 0.6093190  0.3046595

Coefficients of linear discriminants:
                             LD1
ID                 -8.019334e-06
Age                -4.011110e-02
Experience          4.326307e-02
Income              2.073969e-02
Family              2.290416e-01
CCAvg               6.345696e-02
Education           5.545478e-01
Mortgage            7.692707e-04
Securities.Account -4.627287e-01
CD.Account          2.336658e+00
Online             -2.333846e-01
CreditCard         -2.999862e-01

In [11]:
DiscrimPred <- predict(DiscrimFit, newdata=test.df)

In [18]:
summary(DiscrimPred$class)

0    1 
1849  151

In [19]:
confusionMatrix(DiscrimPred$class, test.df$Personal.Loan)

Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1767   82
         1   32  119
                                          
               Accuracy : 0.943           
                 95% CI : (0.9319, 0.9528)
    No Information Rate : 0.8995          
    P-Value [Acc > NIR] : 2.050e-12       
                                          
                  Kappa : 0.6456          
                                          
 Mcnemar's Test P-Value : 4.448e-06       
                                          
            Sensitivity : 0.9822          
            Specificity : 0.5920          
         Pos Pred Value : 0.9557          
         Neg Pred Value : 0.7881          
             Prevalence : 0.8995          
         Detection Rate : 0.8835          
   Detection Prevalence : 0.9245          
      Balanced Accuracy : 0.7871          
                                          
       'Positive' Class : 0               
                        